## Imports & Data read

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import pyarrow.parquet as pq
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
from sklearn.metrics import r2_score, mean_squared_error
import warnings
warnings.filterwarnings('ignore')


In [53]:
path_data_full = r"data/projet 3A/data_merged_20250922.parquet"

table = pq.read_table(path_data_full)
df = table.to_pandas()
df.shape 


(1726886, 957)

### Data preprocess


In [ ]:
cols_explicatives = [
    "annee",
    "codecommune",
    "pvotepreviouspvoteD",
    "pvotepvoteD",


    "popcommunes/pop",
    "popcommunesvbbm/vbbm",
    "agesexcommunes/prop014",
    "agesexcommunes/prop60p",
    "agesexcommunes/perage",
    "diplomescommunes/pbac",
    "diplomescommunes/psup",
    "diplomescommunes/nodip",
    "cspcommunes/pouvr",
    "cspcommunes/pcadr",
    "cspcommunes/pchom",
    "revcommunes/revratio",
    "rsacommunes/perrsa",
    "capitalimmobiliercommunes/prixm2ratio",
    "naticommunes/pimmigre",
]

X = df[cols_explicatives]
y_par = df["pvoteppar"]
y_G = df["pvotepvoteG"]
y_D = df["pvotepvoteD"]

#### Traitement X

In [74]:
# Toutes les colonnes comptent des valeurs manquantes.
# Un objectif dans un premier plan serait d'étudier des techniques ML/Statistiques permettant de combler ces données manquantes.
# Pour l'instant nous étudierons la sous partie du dataset sans valeurs manquantes. 
X.isna().sum()

annee                                          0
codecommune                                    0
pvotepreviouspvoteD                       114963
pvotepvoteD                                    0
popcommunes/pop                            63610
popcommunesvbbm/vbbm                       63623
agesexcommunes/prop014                    802634
agesexcommunes/prop60p                    802634
agesexcommunes/perage                       6265
diplomescommunes/pbac                     802722
diplomescommunes/psup                     802722
diplomescommunes/nodip                    802335
cspcommunes/pouvr                         804365
cspcommunes/pcadr                         804365
cspcommunes/pchom                         805083
revcommunes/revratio                           5
rsacommunes/perrsa                       1588980
capitalimmobiliercommunes/prixm2ratio    1594729
naticommunes/pimmigre                    1441108
dtype: int64

In [ ]:
# Nous n'avons plus accès à 90% du dataset.
# Ce n'est pas envisageable de produire des résulats avec cette sous-partie, pour l'instant utilisons là pour analyser la structure du dataset. 

X_no_na = X.dropna()
X_no_na.shape

(130120, 19)

In [99]:
pd.to_pickle(X_no_na, "data/data_no_na.pkl")

#### Traitement y (label)

In [76]:
def data_process(y: pd.Series, X: pd.DataFrame) -> pd.Series:
    mask = ~(y.isna() | X.isna().any(axis=1))
    return y[mask]

y_par_no_na = data_process(y_par, X)
y_G_no_na = data_process(y_G, X)
y_D_no_na = data_process(y_D, X)


#### Sampling

In [77]:
size_sample = int(1e4)
index_sample = X_no_na.sample(size_sample, random_state=42).index

X_sample = X_no_na.loc[index_sample]

y_par_sample = y_par_no_na.loc[index_sample]
y_G_sample = y_G_no_na.loc[index_sample]
y_D_sample = y_D_no_na.loc[index_sample]


In [78]:
## Etude de la distribution de "annee" dans notre mini-dataset
## On voit que seules les élections présidentielle récente n'ont pas de données manquantes parmi les colonnes selectionnées.

X_sample["annee"].value_counts()

annee
2017    5036
2022    4964
Name: count, dtype: int64

### Etude du poids des features dans la prédiction y pour 2022 et 2017 séparemment.

In [94]:
def train_test_model_rf(X, y, n_estimators=600, min_samples_leaf=2, max_depth=6, min_samples_split=4):
    X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.15,
    random_state=42
    )

    rf = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        min_samples_split=min_samples_split,
        n_jobs=-1,
        bootstrap=True,
        random_state=42
    )

    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_test)

    ### Métriques classiques d'évaluation du modèle
    print("R2:", r2_score(y_test, y_pred))
    print("RMSE:", mean_squared_error(y_test, y_pred) ** 0.5)
    print(f"Std de y: {np.std(y_test):.3f} --- Moyenne de y: {np.mean(y_test):.3f}")

    ### Calcul de l'importance par permutation des features
    perm = permutation_importance(
    rf, X_test, y_test,
    n_repeats=15,
    random_state=42,
    n_jobs=-1
    )

    pi = pd.DataFrame({
        'feature': X.columns,
        'importance_mean': perm.importances_mean,
        'importance_std': perm.importances_std
    }).sort_values('importance_mean', ascending=False)

    print("Permutation Importance: ", pi)
    return rf, pi



index_2017 = X_sample[X_sample["annee"]==2017].index 
index_2022 = X_sample[X_sample["annee"]==2022].index 

X_sample_2022 = X_sample.loc[index_2022]
X_sample_2017 = X_sample.loc[index_2017]


y_D_sample_2017 = y_D_sample.loc[index_2017]
y_D_sample_2022 = y_D_sample.loc[index_2022]


### ---------- Model fit & test -----------------

cols_to_drop = ['pvotepreviouspvoteD', 'annee', 'pvotepvoteD']
# Pour l'instant on s'attardera sur les votes droites.
print("Résultats 2022: \n")
rf_22, pi_22 = train_test_model_rf(X_sample_2022.drop(cols_to_drop, axis=1), y_D_sample_2022)
print("\n---------------------------------------------------------\n")
print("Résultats 2017: \n")
rf_17, pi_17 = train_test_model_rf(X_sample_2017.drop(cols_to_drop, axis=1), y_D_sample_2017)


Résultats 2022: 

R2: 0.22049885067704378
RMSE: 0.10310729941174562
Std de y: 0.117 --- Moyenne de y: 0.346


/Users/khlil/miniconda3/envs/projet3A/lib/python3.10/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/Users/khlil/miniconda3/envs/projet3A/lib/python3.10/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/Users/khlil/miniconda3/envs/projet3A/lib/python3.10/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain

Permutation Importance:                                    feature  importance_mean  importance_std
0                             codecommune         0.116320        0.017465
6                   diplomescommunes/pbac         0.033916        0.007365
14  capitalimmobiliercommunes/prixm2ratio         0.028123        0.008940
15                  naticommunes/pimmigre         0.026620        0.005830
7                   diplomescommunes/psup         0.016690        0.003073
1                         popcommunes/pop         0.014930        0.001959
12                   revcommunes/revratio         0.014275        0.003075
13                     rsacommunes/perrsa         0.010157        0.001911
5                   agesexcommunes/perage         0.008067        0.004103
8                  diplomescommunes/nodip         0.003618        0.001326
3                  agesexcommunes/prop014         0.002324        0.001209
10                      cspcommunes/pcadr         0.002163        0.000658


In [24]:
pi_17.sort_values(by="importance_mean", ascending=False)

,feature,importance_mean,importance_std
7,diplomescommunes/psup,0.055082,0.005932
0,codecommune,0.042204,0.008463
6,diplomescommunes/pbac,0.009152,0.007929
1,popcommunes/pop,0.006782,0.004030
5,agesexcommunes/perage,0.004047,0.010368
4,agesexcommunes/prop60p,0.002095,0.008118
15,naticommunes/pimmigre,0.001969,0.005984
8,diplomescommunes/nodip,0.000562,0.001940
2,popcommunesvbbm/vbbm,0.000240,0.000320
9,cspcommunes/pouvr,-0.000619,0.003940


In [11]:
pi_22.sort_values(by="importance_mean", ascending=False)


,feature,importance_mean,importance_std
5,diplomescommunes/pbac,0.041102,0.008041
13,capitalimmobiliercommunes/prixm2ratio,0.035650,0.005026
4,agesexcommunes/perage,0.021165,0.008811
12,rsacommunes/perrsa,0.020803,0.006516
14,naticommunes/pimmigre,0.017045,0.006107
0,popcommunes/pop,0.010241,0.006363
6,diplomescommunes/psup,0.009966,0.002021
8,cspcommunes/pouvr,0.009603,0.002535
3,agesexcommunes/prop60p,0.007855,0.004208
10,cspcommunes/pchom,0.005248,0.001948


**Commentaires Permutation Importance**

Grâce à la "permutation imortance", nous répondons à la question suivante pour chaque feature: "À quel point est-ce que le modèle se dégrade lorsque nous méleangeons au hasard les donneés de la feature?"
Nous arrivons ainsi à mieux déceler l'impact de chaque feature. 



**Commentaire premier modèle**

Nous avons pour les deux cas un RMSE autour de 0.1 avec y qui se situe etre 0 et 1 avec un écart-type de 0.12. 
Ainsi, nous n'expliquons qu'à peine 10% de la variance de y avec notre modèle, un R2 bas confirme cet hypothèse. Cela implique que 15 features sur une année peinent à expliquer les résultats d'élections, ce qui est prévisible. Il y'a beaucoup plus de facteurs qui entre en jeu et nous n'avons pas utiliser les caractéristiques temporelles de notre dataset. 

### Entraînement du même modèle sur 2017 et une partie de 2022 pour prédire 2022.


In [ ]:
# Only 770 code commune incommon 
X_sample_2022.shape, X_sample_2017.shape, X_sample_2022.merge(X_sample_2017, on="codecommune", how="inner").shape

((4964, 19), (5036, 19), (765, 37))

In [80]:
cols_for_delta = [ 
    "popcommunes/pop",
    "popcommunesvbbm/vbbm",
    "agesexcommunes/prop014",
    "agesexcommunes/prop60p",
    "agesexcommunes/perage",
    "diplomescommunes/pbac",
    "diplomescommunes/psup",
    "diplomescommunes/nodip",
    "cspcommunes/pouvr",
    "cspcommunes/pcadr",
    "cspcommunes/pchom",
    "revcommunes/revratio",
    "rsacommunes/perrsa",
    "capitalimmobiliercommunes/prixm2ratio",
    "naticommunes/pimmigre",
]

In [89]:

codes_communes_inters = np.intersect1d(
    X_sample_2022["codecommune"].values,
    X_sample_2017["codecommune"].values,
)

X_2022_tmp = X_sample_2022.copy()

# 3. Aligner 2017 et 2022 par codecommune
X22 = X_2022_tmp.set_index("codecommune").loc[codes_communes_inters]
X17 = X_sample_2017.set_index("codecommune").loc[codes_communes_inters]

new_df = {}
for col in cols_for_delta:
    name = f"delta_{col}"
    new_df[name] = X22[col] - X17[col]

delta_df = pd.DataFrame(new_df)

X22.reset_index(drop=True, inplace=True)         
delta_df.reset_index(drop=True, inplace=True)

X_augmented_2022 = pd.concat([X22, delta_df], axis=1)


In [90]:
X_augmented_2022.dropna(inplace=True)

In [ ]:
rf_chrono, pi_chrono = train_test_model_rf(X_augmented_2022.drop(["pvotepvoteD", "annee"], axis=1), 
                                           X_augmented_2022["pvotepvoteD"],
                                           n_estimators=300,
                                           min_samples_leaf=4,
                                           min_samples_split=4,
                                           max_depth=6,
                                           )

R2: 0.72879749197773
RMSE: 0.05959663138057069
Std de y: 0.114 --- Moyenne de y: 0.348
Permutation Importance:                                          feature  importance_mean  \
0                           pvotepreviouspvoteD     1.302500e+00   
26                      delta_cspcommunes/pchom     1.623476e-03   
1                               popcommunes/pop     1.560009e-03   
29  delta_capitalimmobiliercommunes/prixm2ratio     1.479827e-03   
18                 delta_agesexcommunes/prop014     1.303286e-03   
28                     delta_rsacommunes/perrsa     1.301196e-03   
9                             cspcommunes/pouvr     1.273303e-03   
14        capitalimmobiliercommunes/prixm2ratio     1.253520e-03   
10                            cspcommunes/pcadr     9.675228e-04   
21                  delta_diplomescommunes/pbac     8.965209e-04   
22                  delta_diplomescommunes/psup     8.673962e-04   
5                         agesexcommunes/perage     7.801726e-04   
25  

In [ ]:
rf_chrono_bis, pi_chrono_bis = train_test_model_rf(X_augmented_2022.drop(["pvotepvoteD", "annee", "pvotepreviouspvoteD"], axis=1), 
                                           X_augmented_2022["pvotepvoteD"],
                                           n_estimators=500,
                                           min_samples_leaf=4,
                                           min_samples_split=4,
                                           max_depth=6,
                                           )

R2: 0.05335286803834882
RMSE: 0.11134453951597233
Std de y: 0.114 --- Moyenne de y: 0.348
Permutation Importance:                                          feature  importance_mean  \
5                         diplomescommunes/pbac     9.844400e-02   
4                         agesexcommunes/perage     3.385168e-02   
14                        naticommunes/pimmigre     1.606722e-02   
2                        agesexcommunes/prop014     1.029383e-02   
3                        agesexcommunes/prop60p     8.516227e-03   
22                 delta_diplomescommunes/nodip     6.725872e-03   
12                           rsacommunes/perrsa     5.763571e-03   
25                      delta_cspcommunes/pchom     5.178967e-03   
7                        diplomescommunes/nodip     3.366490e-03   
28  delta_capitalimmobiliercommunes/prixm2ratio     3.352811e-03   
13        capitalimmobiliercommunes/prixm2ratio     2.877072e-03   
0                               popcommunes/pop     2.444792e-03   
2

**L'impact de l'ajout de "pvotepreviouspvoteD" est très important du point de vue de la RMSE et du R2 score.**

L'ajout des colonnes delta a nettement amélioré le R2 score par rapport aux prédictions en utilisant que les données d'une année. Ainsi, la variation des indicateurs éco-sociaux influent sur les performances du modèles.
On notera l'importance que prends le vote de l'année précédente sur le vote actuel. 
